# Pennylane: I. Introduction to Quantum Computing
## I.10. What Did You Expect?

In [1]:
# preparation
import numpy as np
import pennylane as qml
import altair as alt
import pandas as pd

### Codercise I.10.1 - Measurement of the PauliY observable

To compute expectation values, we can use `qml.expval()` and specify the observable to be measured. Common choices are Pauli-X, Pauli-Y, and Pauli-Z operations. The set of possible eigenvalues or any Pauli-based expectation values have a range of $[1, -1]$.

**Task**: Design and run the following circuit where $\langle Y\rangle$ indicates measurement of the Pauli-Y observable.

<img height="50%" width="50%" src="https://assets.cloud.pennylane.ai/codebook/exercise_i101.svg"/>

**Solution**: We need to apply the new function.

In [2]:
dev = qml.device("default.qubit", wires=1)


@qml.qnode(dev)
def circuit():
    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE CIRCUIT IN THE PICTURE AND MEASURE PAULI Y
    qml.RX(np.pi/4, wires=0)
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=0)
    return qml.expval(qml.PauliY(wires=0))

print(circuit())

-0.7071067811865471


### Codercise I.10.2 - Setting up the number of experiment shots

While executing the experiment 'N' times, we can use the parameter `shots` in `qml.device()` declaration.

**Task**: For each shot, initiate a device and create a QNode for the above circuit. What happens to the expectation value as the number of shots gets larger.

**Solution**: We need to insert the parameter `shot` to the device initiation and loop it.

In [3]:
# An array to store your results
shot_results = []

# Different numbers of shots
shot_values = [100, 1000, 10000, 100000, 1000000]

for shots in shot_values:
    ##################
    # YOUR CODE HERE #
    ##################

    # CREATE A DEVICE, CREATE A QNODE, AND RUN IT
    dev = qml.device("default.qubit", wires=1, shots=shots)
    @qml.qnode(dev)
    def circuit():
        qml.RX(np.pi/4, wires=0)
        qml.Hadamard(wires=0)
        qml.PauliZ(wires=0)
        return qml.expval(qml.PauliY(wires=0))

    # STORE RESULT IN SHOT_RESULTS ARRAY
    shot_results.append(circuit())

print(qml.math.unwrap(shot_results))

[-0.76, -0.706, -0.699, -0.70564, -0.707064]


### Codercise I.10.3 - Evaluating the samples


We can get a sample directly with `qml.sample()`. The difference between `qml.expval()` and `qml.sample()` is that the former shows a *weighted average* value to be expected after all possible outcomes, while the latter evaluates each outcome and returns a list of outcomes (eigenvalues). The former cares about what the outcome would be with larger shots, while the latter focuses on each instance.
- `qml.expval()` -> int --> -0.70710678...
- `qml.sample()` -> List[int] --> [1, -1, -1, 1, 1, ...]

We can also indirectly obtain the expectation value via a sample. The method is similar to taking a *weighted average*. Let number of 1s be '$n_1$', numbers of -1s be '$n_2$', and the number of shots be 's'. Then, we can find the expectation value with-

$$\langle Y\rangle=\frac{1(n_1) + (-1) (n_2)}{s}$$

**Task**: Replace `qml.expval()` with `qml.sample()` for the previous circuit. Write a function to compute an estimate of the expectation value based on the samples.

**Solution**: We need to implement the equation. The sample is analyzed for its content, i.e., the number of times it contains 1s and -1s.

In [4]:
dev = qml.device("default.qubit", wires=1, shots=100000)


@qml.qnode(dev)
def circuit():
    qml.RX(np.pi / 4, wires=0)
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=0)

    ##################
    # YOUR CODE HERE #
    ##################

    # RETURN THE MEASUREMENT SAMPLES OF THE CORRECT OBSERVABLE

    return qml.sample(qml.PauliY(wires=0))


def compute_expval_from_samples(samples):
    """Compute the expectation value of an observable given a set of
    sample outputs. You can assume that there are two possible outcomes,
    1 and -1.

    Args:
        samples (np.array[float]): 100000 samples representing the results of
            running the above circuit.

    Returns:
        float: the expectation value computed based on samples.
    """

    estimated_expval = 0

    ##################
    # YOUR CODE HERE #
    ##################

    # USE THE SAMPLES TO ESTIMATE THE EXPECTATION VALUE
    n1, n2 = 0, 0 # initiate respective counts
    for sample in samples:
        if sample == 1:
            n1 += 1 # number of 1s
        elif sample == -1:
            n2 += 1 # number of -1s
    estimated_expval = ((1 * n1) + (-1 * n2)) / len(samples)
    return estimated_expval


samples = circuit()
print(compute_expval_from_samples(samples))

-0.70434


### Plotting Function
Before moving on, we need to implement the plotting function to use.

In [5]:
# define pennylane plotter function
def plotter(shot_vals, results_experiment, results_scaling):
    """Plot the value of the output

    Args:
        shot_vals (np.array[int]): Angles for the x axis.
        results_experiment (np.array[float]): Variances from the experiments
        results_scaling (np.array[float]): Variances computed from scaling function.
    """

    df = pd.DataFrame(
        {
            "shots": shot_vals,
            "experiment": results_experiment,
            "function": results_scaling,
        }
    )

    plot = (
        alt.Chart(df)
        .transform_fold(["experiment", "function"], ["var_type", "variance"])
        .mark_point()
        .encode(x="shots:Q", y="variance:Q", color="var_type:N")
    )

    return plot

### Codercise I.10.4 - The variance of sample measurements

**Task**: The accuracy of the expectation value depends on the number of shots. How does the variance scale with the number of shots?

**Solution**: 
1. The first function takes a specific value of shots (*n_shots*) and repeats that `shots` for 100 (*n_trials*) times.
   - For each `shots`, there will be a separate loop.
     - If the value of shots is 1000, that 1000-shot evaluation will run 100 times.
   - The first function is called until the possible `shots` value is exhausted.
   - These values are provided with a list of `shots`.
   - The `np.var()` function can be used to compute the variance.
2. The second function estimates the variance with `shots`.
   - Typically, the variance is $\frac{1}{n}$ where 'n' is the number of shots.

In [6]:
def variance_experiment(n_shots):
    """Run an experiment to determine the variance in an expectation
    value computed with a given number of shots.

    Args:
        n_shots (int): The number of shots

    Returns:
        float: The variance in expectation value we obtain running the
        circuit 100 times with n_shots shots each.
    """

    # To obtain a variance, we run the circuit multiple times at each shot value.
    n_trials = 100

    ##################
    # YOUR CODE HERE #
    ##################

    # CREATE A DEVICE WITH GIVEN NUMBER OF SHOTS
    dev = qml.device("default.qubit", wires=1, shots=n_shots)
    # DECORATE THE CIRCUIT BELOW TO CREATE A QNODE
    @qml.qnode(dev)
    def circuit():
        qml.Hadamard(wires=0)
        return qml.expval(qml.PauliZ(wires=0))

    # RUN THE QNODE N_TRIALS TIMES AND RETURN THE VARIANCE OF THE RESULTS
    results = [circuit() for _ in range(n_trials)]
    variance = np.var(results)
    return variance


def variance_scaling(n_shots):
    """Once you have determined how the variance in expectation value scales
    with the number of shots, complete this function to programmatically
    represent the relationship.

    Args:
        n_shots (int): The number of shots

    Returns:
        float: The variance in expectation value we expect to see when we run
        an experiment with n_shots shots.
    """

    estimated_variance = 0

    ##################
    # YOUR CODE HERE #
    ##################

    # ESTIMATE THE VARIANCE BASED ON SHOT NUMBER
    estimated_variance = 1 / n_shots
    return estimated_variance


# Various numbers of shots; you can change this
shot_vals = [10, 20, 40, 100, 200, 400, 1000, 2000, 4000]

# Used to plot your results
results_experiment = [variance_experiment(shots) for shots in shot_vals]
results_scaling = [variance_scaling(shots) for shots in shot_vals]
plot = plotter(shot_vals, results_experiment, results_scaling)

In [7]:
plot

alt.Chart(...)

This notebook is done by `Myanmar Youths` for `Womanium Quantum + AI 2024` program.
- <a href="https://www.linkedin.com/in/la-wun-nannda-b047681b5/"><u>La Wun Nannda</u></a>
- <a href="https://www.linkedin.com/in/chit-zin-win-46a2a3263/"><u>Chit Zin Win</u></a>